In [1]:
pip install --upgrade "ibm-watson>=4.5.0" --quiet

ERROR: kubernetes 10.1.0 has requirement pyyaml~=3.12, but you'll have pyyaml 5.3.1 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [9]:
#Pandas for data handling
import pandas as pd

#Numpy for mathematical calculations
import numpy as np

#Time class to measure the time taken
from datetime import datetime

from tqdm import tqdm

In [3]:
from ibm_watson import ToneAnalyzerV3
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

version = '2017-09-21'
apikey = 'XYZ'

authenticator = IAMAuthenticator(apikey)
tone_analyzer = ToneAnalyzerV3(
    version= version,
    authenticator=authenticator
)

tone_analyzer.set_service_url('https://api.us-east.tone-analyzer.watson.cloud.ibm.com/instances/XYZ')

In [11]:
df = pd.read_excel('../input/cnn-covid19-news-articles/CNN_coronavirus1.xlsx')
df = df[['headline', 'published_date', 'content']]
df.head()

,headline,published_date,content
0,Latino communities in the US devastated by Cov...,"May 9, 2020",In Chicago's Littl...
1,How Covid-19 misinformation is still going viral,"May 8, 2020",Despite pledges fr...
2,3 children die of syndrome potentially linked ...,"May 9, 2020",Three New York chi...
3,Spanish prime minister fails to prove existenc...,"May 9, 2020",In a nationally te...
4,FDA chief self-quarantines after coming in con...,"May 9, 2020","Dr. Stephen Hahn, ..."


In [12]:
#Get the current time on the clock
time_start = datetime.now()

In [13]:
import json

text = 'Team, I know that times are tough! Product '\
    'sales have been disappointing for the past three '\
    'quarters. We have a competitive product, but we '\
    'need to do a better job of selling it!'\
    'I will kill you.'

tone_analysis = tone_analyzer.tone(
    {'text': text},
    content_type='text/plain').get_result()
print(tone_analysis)

{'document_tone': {'tones': [{'score': 0.706084, 'tone_id': 'sadness', 'tone_name': 'Sadness'}, {'score': 0.793256, 'tone_id': 'analytical', 'tone_name': 'Analytical'}]}, 'sentences_tone': [{'sentence_id': 0, 'text': 'Team, I know that times are tough!', 'tones': [{'score': 0.801827, 'tone_id': 'analytical', 'tone_name': 'Analytical'}]}, {'sentence_id': 1, 'text': 'Product sales have been disappointing for the past three quarters.', 'tones': [{'score': 0.771241, 'tone_id': 'sadness', 'tone_name': 'Sadness'}, {'score': 0.687768, 'tone_id': 'analytical', 'tone_name': 'Analytical'}]}, {'sentence_id': 2, 'text': 'We have a competitive product, but we need to do a better job of selling it!I will kill you.', 'tones': []}]}


In [14]:
print(tone_analysis['document_tone']['tones'])

[{'score': 0.706084, 'tone_id': 'sadness', 'tone_name': 'Sadness'}, {'score': 0.793256, 'tone_id': 'analytical', 'tone_name': 'Analytical'}]


In [15]:
for i in tone_analysis['document_tone']['tones']:
    #Append the attributes to the data
    print(i['tone_name'], i['score']) 

Sadness 0.706084
Analytical 0.793256


In [ ]:
for index, review in tqdm(df['content'].iteritems()):
    
    #Pass a single article to Tone Analyzer:
    json_output = tone_analyzer.tone(review, content_type='text/plain').get_result()    
    
    #Append the results to the original dataset
    for i in json_output['document_tone']['tones']:
        #Append the attributes to the data
        df.at[index, i['tone_name']] = i['score']

In [19]:
df.replace(np.NaN, 0, inplace=True)

In [20]:
df.head()

,headline,published_date,content,Joy,Sadness,Analytical,Tentative,Confident,Fear,Anger
0,Latino communities in the US devastated by Cov...,"May 9, 2020",In Chicago's Littl...,0.526852,0.595025,0.000000,0.000000,0.000000,0.0,0.0
1,How Covid-19 misinformation is still going viral,"May 8, 2020",Despite pledges fr...,0.530503,0.544207,0.705582,0.753454,0.000000,0.0,0.0
2,3 children die of syndrome potentially linked ...,"May 9, 2020",Three New York chi...,0.000000,0.000000,0.929825,0.779803,0.000000,0.0,0.0
3,Spanish prime minister fails to prove existenc...,"May 9, 2020",In a nationally te...,0.511291,0.000000,0.721050,0.632847,0.000000,0.0,0.0
4,FDA chief self-quarantines after coming in con...,"May 9, 2020","Dr. Stephen Hahn, ...",0.591562,0.000000,0.000000,0.000000,0.802229,0.0,0.0


In [54]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7223 entries, 0 to 7222
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   headline        7223 non-null   object 
 1   published_date  7223 non-null   object 
 2   content         7223 non-null   object 
 3   Joy             6287 non-null   float64
 4   Sadness         6515 non-null   float64
 5   Analytical      6681 non-null   float64
 6   Tentative       5968 non-null   float64
 7   Confident       5302 non-null   float64
 8   Fear            5456 non-null   float64
 9   Anger           5275 non-null   float64
dtypes: float64(7), object(3)
memory usage: 564.4+ KB


In [55]:
df.to_excel('CNN_articles_tones.xlsx')